### Quick Run Block

## Code

In [1]:
import sys
from pathlib import Path
import os

sys.path.append(str(Path.cwd().parent.parent))
print(str(Path.cwd().parent.parent))
print("Current working directory: ", os.getcwd())

from ModelDevelopment.CentralPipeline import CentralPipeline
from ModelDevelopment.ImageBatchPipeline import ImageBatchPipeline
from DataProcessing.DataPreProcessing import DataPaths

c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition
Current working directory:  c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\ModelDevelopment\experiments


In [2]:
# TODO: Add tracklets_override to pass a list of tracklets to process; tracklets_override=["8", "9", "10"]
# NOTE: It is recommended you delete the entire processed_data/{challenge/test/train} before running this to 100% avoid issues with old data.
# Furthermore, you should always restart your kernel before every new run because sometimes there are problems with paths
pipeline = CentralPipeline(
  num_tracklets=1210,
  #num_images_per_tracklet=1,
  input_data_path=DataPaths.TEST_DATA_DIR.value,
  output_processed_data_path=DataPaths.PROCESSED_DATA_OUTPUT_DIR_TEST.value,
  common_processed_data_dir=DataPaths.COMMON_PROCESSED_OUTPUT_DATA_TEST.value,
  gt_data_path=DataPaths.TEST_DATA_GT.value,
  single_image_pipeline=False,
  display_transformed_image_sample=False, # NOTE: DO NOT USE. Code is parallelized so we cannot show images anymore. Code breaks, but first one will show if True.
  num_image_samples=1,
  use_cache=False, # Better set to false, not stable
  suppress_logging=False,
  num_workers=4
  )

2025-03-22 21:10:15 [INFO] DataPreProcessing initialized. Universe of available data paths:
2025-03-22 21:10:15 [INFO] ROOT_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted
2025-03-22 21:10:15 [INFO] TEST_DATA_GT: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\test_gt.json
2025-03-22 21:10:15 [INFO] TRAIN_DATA_GT: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\train_gt.json
2025-03-22 21:10:15 [INFO] TEST_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\images
2025-03-22 21:10:15 [INFO] TRAIN_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images
2025-03-22 21:10:15 [INFO] CHALLENGE_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\challenge\images
2

In [3]:
pipeline.run_soccernet(
  run_soccer_ball_filter=True,
  generate_features=True,
  run_filter=True,
  run_legible=True,
  run_legible_eval=True,
  run_pose=False,
  run_crops=False,
  run_str=False,
  run_combine=False,
  run_eval=False)

2025-03-22 21:10:15 [INFO] Running the SoccerNet pipeline.
2025-03-22 21:10:15 [INFO] Using single-process GPU mode (no CPU multiprocessing).


Processing tracklets (CUDA-Only):   0%|          | 0/1210 [00:00<?, ?it/s]

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
